In [2]:
import csv
import pandas as pd
from elasticsearch import Elasticsearch
from imdb import IMDb

In [3]:
# Get data frame with data from movies
df = pd.read_csv('movies.csv', sep=',')

In [4]:
# Save only western movies to new dataframe
western_movies = df.loc[df['genres'] == 'Western']

In [5]:
# Get movie IDs
western_titles = western_movies.as_matrix(columns=df.columns[1:2])

In [6]:
ia = IMDb()
movieIDs = []
for title in western_titles:
    movies = ia.search_movie(title.item(0))
    if(len(movies) > 0):
        movieIDs.append(movies[0].movieID)

In [7]:
es = Elasticsearch()

In [8]:
es.indices.create(index="movie-index", ignore=400)

{u'error': {u'index': u'movie-index',
  u'index_uuid': u'WkykToMrT9GaNj7sqywbRw',
  u'reason': u'index [movie-index/WkykToMrT9GaNj7sqywbRw] already exists',
  u'root_cause': [{u'index': u'movie-index',
    u'index_uuid': u'WkykToMrT9GaNj7sqywbRw',
    u'reason': u'index [movie-index/WkykToMrT9GaNj7sqywbRw] already exists',
    u'type': u'index_already_exists_exception'}],
  u'type': u'index_already_exists_exception'},
 u'status': 400}

In [18]:
ia = IMDb()
for i in range(1, len(movieIDs)):
    movie = ia.get_movie(movieIDs[i])
    es.index(index='movie-index', doc_type='movie', body={
        'id': i,
        'title': movie.get('title'),
        'plot': movie.get('plot outline')
    })
        

In [20]:
es.search(index='movie-index', q='title:"Dakota"')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'AVymQpthE-HKshLZMdsI',
    u'_index': u'movie-index',
    u'_score': 5.776349,
    u'_source': {u'id': 31,
     u'plot': u'In 1871 Dakota, two crooked businessmen oppose the local wheat farmers and the railroad development, in order to control the town of Fargo.',
     u'title': u'Dakota'},
    u'_type': u'title'}],
  u'max_score': 5.776349,
  u'total': 1},
 u'timed_out': False,
 u'took': 335}